# check gpu resources

In [ ]:
# gpu resources
! nvidia-smi

# running ManTraNet on OpenMFC dataset in Parallel

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import cv2
import requests
import sys
import image_slicer
import time
import tensorflow as tf
import pandas as pd
import ipyparallel as ipp

from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from matplotlib import pyplot
from PIL import UnidentifiedImageError
from datetime import datetime 

In [ ]:
# ManTraNet paths
manTraNet_root = './'
manTraNet_srcDir = os.path.join( manTraNet_root, 'src' )
sys.path.insert( 0, manTraNet_srcDir )
manTraNet_modelDir = os.path.join( manTraNet_root, 'pretrained_weights' )

# setup dataset paths

In [ ]:
# set the part #'s
dataset_part_nums = [1, 2, 3, 4]
datasets = []

In [ ]:
# OpenMFC dataset
mfc_data = os.path.join(manTraNet_root, 'openmfc_data')
openmfc_2020 = os.path.join(mfc_data, 'OpenMFC2020')

# p1 for index
openmfc_2020_p1 = os.path.join(openmfc_2020, 'OpenMFC20_Image_Ver1-part001of27')
openmfc_2020_p1_indexes = os.path.join(openmfc_2020_p1, 'indexes')

# setup part list
for part_num in dataset_part_nums:
    if (part_num < 10):
        openmfc_2020_part = os.path.join(openmfc_2020, f'OpenMFC20_Image_Ver1-part00{part_num}of27')
    else:
        openmfc_2020_part = os.path.join(openmfc_2020, f'OpenMFC20_Image_Ver1-part0{part_num}of27')

    openmfc_2020_part_probe = os.path.join(openmfc_2020_part, 'probe')
    openmfc_2020_part_mask_out = os.path.join(openmfc_2020_part, 'predicted_masks')
    datasets.append(
        (openmfc_2020_part, openmfc_2020_part_probe, os.listdir(openmfc_2020_part_probe), openmfc_2020_part_mask_out)
    )

datasets

# load dataset description

In [ ]:
imdl_index_path = os.path.join(openmfc_2020_p1_indexes, 'OpenMFC20_Image-IMDL-index.csv')

In [ ]:
df = pd.read_csv(imdl_index_path, sep='|')

In [ ]:
df.head()

# ipyparallel engine and view setup

In [ ]:
! ipcluster start -n 4 --daemonize

In [ ]:
client = ipp.Client()

In [ ]:
client.ids

In [ ]:
view = client[:]

# parallel function setup

In [ ]:
def p_prediction(dataset):
    # break dataset for process out into individual elements of created Tuple
    part_path, part_probe_path, probes, mask_out_path = dataset    
    # load pretrained mantranet model
    import modelCore
    manTraNet = modelCore.load_pretrain_model_by_index( 4, manTraNet_modelDir )
    # import functions for prediction
    import lib
    # run prediction on probes and return result
    
    start = time.time()
    failures = []

    for sample in part_probe:
        try:
            sample_probe = part_probe_path + '/' + sample
            # manipulation detection using slicing
            mask, ptime = lib.slice_and_decode( sample_probe, manTraNet )
            mask_filename = mask_out_path + '/'+ os.path.splitext(sample)[0] + '_mask' + '.png'
            # save mask to mask path
            mask.save(fp=mask_filename, format='PNG')
        except UnidentifiedImageError as err:
            failures.append((sample, err))
        except tf.errors.ResourceExhaustedError as err:
            failures.append((sample, err))
        except:
            failures.append((sample, sys.exc_info()[0]))

    time_msg = f'all probes completed in {time.time() - start} seconds'
    failure_count_msg = f'failure count: {len(failures)}'
    df = pd.DataFrame(failures, columns=['FileName', 'Error'])
    df.to_csv(os.path.join(part_path, 'failures.csv'))
    
    return (time_msg, failure_count_msg)

# run in parallel on probes

In [ ]:
results = view.map(p_prediction, datasets)

# cleanup ipyparallel resources

In [ ]:
! ipcluster stop

client.shutdown()
client.close()